# Reconocimiento de Autos y Daño en imágenes

## Introducción

Proyecto para procesar y clasificar las imágenes subidas por los usuarios de La Segunda por medio de la app del celular para inspecciones previas.

El proyecto persigue dos objetivos:
* Detectar si la imágen procesada corresponde a un auto o no
* Detectar si el auto de la imágen se encuentra dañado o no

## Camino recorrido

* Búsqueda de datasets de vehículos sanos y dañados por choques. Se armó un dataset compaginado entre datasets de internet, datos reales obtenidos de la empresa en la cual trabajamos e imágenes seleccionadas de forma manual de la búsqueda de google.
* Evaluación de alternativas de solución. En función de las consultas realizadas a los profesores y de trabajos relacionados encontrados en internet, decidimos utilizar transfer learning con VGG16.

![title](02-guide-how-transfer-learning-v3-06.png)

* Se optó por crear una red diferente para cada objetivo

## Consideraciones comunes a ambas redes

1. Se utiliza transfer learning desde VGG16
    ![title](vgg16.png)
2. Las imágenes se redimensionan a un tamaño de 224x224
3. Se hace un reescalado de las imágenes de 1/255
4. Se aplica data augmentation aleatorio de las siguientes maneras
     - Rotacion de 90 grados
     - Flip horizontal
     - Flip vertical
     - Zoom de 0.2
5. La red 1 se entrenó con aproximadamente 1500 imágenes por clase. La red 2 se entrenó con aproximadamente 800 imágenes por clase

### Red 1 - Clasificación de auto o no

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import math
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras import regularizers

# dimensión de las imágenes.
img_width, img_height = 224, 224

top_model_weights_path = 'bottleneck_model_2_clases.h5'  
train_data_dir = 'C:/data/car-damage/2clases/training'
validation_data_dir = 'C:/data/car-damage/2clases/validation'

In [3]:
def save_bottlebeck_features():
    
    model = applications.VGG16(include_top=False, weights='imagenet')

    datagen = ImageDataGenerator(rescale=1. / 255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rotation_range=90)

    generator = datagen.flow_from_directory(train_data_dir,  
                                             target_size=(img_width, img_height),  
                                             batch_size=batch_size,  
                                             class_mode=None,  
                                             shuffle=False)

    nb_train_samples = len(generator.filenames)
    num_classes = len(generator.class_indices)

    predict_size_train = int(math.ceil(nb_train_samples / batch_size))
    bottleneck_features_train = model.predict_generator(generator, predict_size_train)
    np.save('bottleneck_2_clases_features_train.npy', bottleneck_features_train)

    generator = datagen.flow_from_directory(validation_data_dir,  
                                            target_size=(img_width, img_height),  
                                            batch_size=batch_size,  
                                            class_mode=None,  
                                            shuffle=False)  

    nb_validation_samples = len(generator.filenames)

    predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

    bottleneck_features_validation = model.predict_generator(generator, predict_size_validation)

    np.save('bottleneck_2_clases_features_validation.npy', bottleneck_features_validation)

In [4]:
def train_top_model(number_model):
    datagen_top = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    rotation_range=90)

    generator_top = datagen_top.flow_from_directory(train_data_dir,  
                                                    target_size=(img_width, img_height),  
                                                    batch_size=batch_size,  
                                                    class_mode='categorical',  
                                                    shuffle=False)  

    nb_train_samples = len(generator_top.filenames)  
    num_classes = len(generator_top.class_indices)  

    # load the bottleneck features saved earlier  
    train_data = np.load('bottleneck_2_clases_features_train.npy')  

    # get the class lebels for the training data, in the original order  
    train_labels = generator_top.classes  

    # convert the training labels to categorical vectors  
    train_labels = to_categorical(train_labels, num_classes=num_classes)
    
    generator_top = datagen_top.flow_from_directory(validation_data_dir,  
                                                target_size=(img_width, img_height),  
                                                batch_size=batch_size,  
                                                class_mode=None,  
                                                shuffle=False)  

    nb_validation_samples = len(generator_top.filenames)

    validation_data = np.load('bottleneck_2_clases_features_validation.npy')

    validation_labels = generator_top.classes
    validation_labels = to_categorical(validation_labels, num_classes=num_classes)
    
    model = prepare_model(number_model, train_data.shape[1:])

    checkpointer = ModelCheckpoint(monitor='val_acc', filepath='bottleneck_model_2_clases.h5', verbose=1, save_best_only=True)
    
    history = model.fit(train_data, train_labels,  
                      epochs=epochs,  
                      batch_size=batch_size,  
                      validation_data=(validation_data, validation_labels),
                    callbacks=[checkpointer])  

    #model.save_weights(top_model_weights_path)  

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size=batch_size, verbose=1)

    print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
    print("[INFO] Loss: {}".format(eval_loss))
    
    plt.figure(1)  

    # summarize history for accuracy  

    plt.subplot(211)  
    plt.plot(history.history['acc'])  
    plt.plot(history.history['val_acc'])  
    plt.title('model accuracy')  
    plt.ylabel('accuracy')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'test'], loc='upper left')  

    # summarize history for loss  

    plt.subplot(212)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def prepare_model(number_model, shape):
    
    if (number_model == 1):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu'))  
        model.add(Dropout(0.5))  
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 2):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 3):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 4):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.RMSprop(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 5):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.8))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.8))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.8))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 6):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.8))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    elif (number_model == 7):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 8):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 9):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 10):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    elif (number_model == 11):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu'))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))  

        model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    elif (number_model == 12):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.RMSprop(lr=0.01, decay=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

    elif (number_model == 13):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.SGD(lr=0.001, decay=0.1, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 14):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.RMSprop(lr=0.001, decay=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    
    elif (number_model == 15):
        model = Sequential()
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    return model

### Creación de los features por medio de la red VGG16

In [ ]:
batch_size = 16
save_bottlebeck_features()

### Entrenamiento de los Top Models

In [ ]:
batch_size = 1024
epochs = 600
num_classes = 2
train_top_model(1)

In [ ]:
train_top_model(2)

In [ ]:
train_top_model(3)

In [ ]:
train_top_model(4)

In [ ]:
epochs = 1200
train_top_model(5)

In [ ]:
epochs = 1000
train_top_model(6)

In [ ]:
train_top_model(7)

In [ ]:
epochs = 600
train_top_model(8)

In [ ]:
train_top_model(9)

In [ ]:
train_top_model(10)

In [ ]:
train_top_model(11)

In [ ]:
train_top_model(12)

In [ ]:
train_top_model(13)

In [ ]:
train_top_model(14)

In [ ]:
train_top_model(15)

### Resultados de cada modelo

Los resultados de cada modelo se encuentran detallados en este [link](https://docs.google.com/spreadsheets/d/15_S33peWDFdnzXx5T0yxGEaGsqcYP6dQLyk6DUqXo5A/edit#gid=0) en la solapa "Auto o No".

##### El modelo seleccionado como el mejor es el 8
- Acc: 0,9922
- Val Acc: 0,9222
- Loss: 1,6056
- Val Loss: 1,7623

### Fine Tunning

In [ ]:
from keras.models import Model
from keras.models import load_model

top_model_weights_path = 'bottleneck_model_2_clases.h5'
epochs = 50
batch_size = 64
datagen_train_fine = ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rotation_range=90)

generator_train_fine = datagen_train_fine.flow_from_directory(train_data_dir,  
                                                target_size=(img_width, img_height),  
                                                batch_size=batch_size,  
                                                class_mode='categorical',  
                                                shuffle=False)

nb_train_samples_fine = len(generator_train_fine.filenames)  
num_classes = len(generator_train_fine.class_indices)  

train_data = generator_train_fine
train_labels = generator_train_fine.classes  
train_labels = to_categorical(train_labels, num_classes=num_classes)

generator_validation_fine = datagen_train_fine.flow_from_directory(validation_data_dir,  
                                                        target_size=(img_width, img_height),  
                                                        batch_size=batch_size,  
                                                        class_mode='categorical',  
                                                        shuffle=False)  

nb_validation_samples_fine = len(generator_validation_fine.filenames)

validation_data = nb_validation_samples_fine
validation_labels = generator_validation_fine.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)


base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))

print(base_model.output_shape[1:])

# El top model con mejor resultado
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))  
top_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))  

top_model.compile(optimizer=optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
        
model_fine = Model(inputs=base_model.input, outputs=top_model(base_model.output))

#Seteo como no entrenables las primeras capas de la red
for layer in model_fine.layers[:15]:
    print(layer.name)
    layer.trainable = False
model_fine.layers[15].name

model_fine.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

print(model_fine.summary())

#model_fine = load_model('MRETZLAF_2CLASES_FINE.hdf5')

checkpointer = ModelCheckpoint(monitor='val_acc', filepath='MRETZLAF_2CLASES_FINE.hdf5', verbose=1, save_best_only=True)

history_fine = model_fine.fit_generator(generator_train_fine,
                                        steps_per_epoch=nb_train_samples_fine // batch_size,
                                        epochs=epochs,
                                        validation_data=generator_validation_fine,
                                        validation_steps=nb_validation_samples_fine // batch_size,
                                        verbose=1,
                                        callbacks=[checkpointer])

plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.show()

plt.plot(history_fine.history['acc'])
plt.plot(history_fine.history['val_acc'])
plt.show()

### Resultados

**Accuracy**
- Acc: 0,9974
- Val Acc: 0,96667
![Accuracy](Fine_acc_red1.png)

**Loss**
- Loss: 0,0147
- Val Loss: 0,2033
![title](Fine_loss_red1.png)

### Testing Data

In [ ]:
test_data_dir = 'C:/data/car-damage/2clases/testing'
test_datagen = ImageDataGenerator(rescale=1. / 255)
flow_test = test_datagen.flow_from_directory(test_data_dir,  target_size=(img_width, img_height),  batch_size=batch_size,class_mode='categorical')
print(model_fine.metrics_names)
model_fine.evaluate_generator(flow_test,5)

- Test Acc: 0,953135
- Test Loss: 0,2075

### Red 2 - Auto dañado o no

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import math
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras import regularizers

#Ruta del dataset
location = 'C:/data/car-damage/damaged_or_not'

#Dimensiones de la imagen
img_width, img_height = 224, 224

train_data_dir = location+'/training'
validation_data_dir = location+'/validation'

# number of epochs to train top model  
epochs = 50
# batch size used by flow_from_directory and predict_generator  
batch_size = 16

In [ ]:
def save_bottlebeck_features():
    
    model = applications.VGG16(include_top=False, weights='imagenet')

    datagen = ImageDataGenerator(rescale=1. / 255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rotation_range=90)

    generator = datagen.flow_from_directory(train_data_dir,  
                                             target_size=(img_width, img_height),  
                                             batch_size=batch_size,  
                                             class_mode=None,  
                                             shuffle=False)

    nb_train_samples = len(generator.filenames)
    num_classes = len(generator.class_indices)

    predict_size_train = int(math.ceil(nb_train_samples / batch_size))
    bottleneck_features_train = model.predict_generator(generator, predict_size_train)
    np.save('bottleneck_damaged_or_not_features_train.npy', bottleneck_features_train)

    generator = datagen.flow_from_directory(validation_data_dir,  
                                            target_size=(img_width, img_height),  
                                            batch_size=batch_size,  
                                            class_mode=None,  
                                            shuffle=False)  

    nb_validation_samples = len(generator.filenames)

    predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

    bottleneck_features_validation = model.predict_generator(generator, predict_size_validation)

    np.save('bottleneck_damaged_or_not_features_validation.npy', bottleneck_features_validation)

In [ ]:
def train_top_model(number_model):
    datagen_top = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    rotation_range=90)

    generator_top = datagen_top.flow_from_directory(train_data_dir,  
                                                    target_size=(img_width, img_height),  
                                                    batch_size=batch_size,  
                                                    class_mode='categorical',  
                                                    shuffle=False)  

    nb_train_samples = len(generator_top.filenames)  
    num_classes = len(generator_top.class_indices)  

    # load the bottleneck features saved earlier  
    train_data = np.load('bottleneck_damaged_or_not_features_train.npy')  

    # get the class lebels for the training data, in the original order  
    train_labels = generator_top.classes  

    # convert the training labels to categorical vectors  
    train_labels = to_categorical(train_labels, num_classes=num_classes)
    
    generator_top = datagen_top.flow_from_directory(validation_data_dir,  
                                                target_size=(img_width, img_height),  
                                                batch_size=batch_size,  
                                                class_mode=None,  
                                                shuffle=False)  

    nb_validation_samples = len(generator_top.filenames)

    validation_data = np.load('bottleneck_damaged_or_not_features_validation.npy')

    validation_labels = generator_top.classes
    validation_labels = to_categorical(validation_labels, num_classes=num_classes)
    
    model = prepare_model(number_model, train_data.shape[1:])

    checkpointer = ModelCheckpoint(monitor='val_acc', filepath='bottleneck_damaged_or_not.h5', verbose=1, save_best_only=True)
    
    history = model.fit(train_data, train_labels,  
                      epochs=epochs,  
                      batch_size=batch_size,  
                      validation_data=(validation_data, validation_labels),
                    callbacks=[checkpointer])  

    #model.save_weights(top_model_weights_path)  

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size=batch_size, verbose=1)

    #print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
    #print("[INFO] Loss: {}".format(eval_loss))
    
    plt.figure(1)  

    # summarize history for accuracy  

    plt.subplot(211)  
    plt.plot(history.history['acc'])  
    plt.plot(history.history['val_acc'])  
    plt.title('model accuracy')  
    plt.ylabel('accuracy')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'test'], loc='upper left')  

    # summarize history for loss  

    plt.subplot(212)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def prepare_model(number_model, shape):
    
    if (number_model == 1):
        model = Sequential()
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 2):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 3):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 4):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.7))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 5):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 6):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.3))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 7):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.3))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
    elif (number_model == 8):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.RMSprop(lr=0.01, decay=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
    
    elif (number_model == 9):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu'))  
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))
        
        model.compile(optimizer=optimizers.RMSprop(lr=0.01, decay=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
    
    elif (number_model == 10):
        model = Sequential()  
        model.add(Flatten(input_shape=shape))  
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(optimizer=optimizers.RMSprop(lr=0.01, decay=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

### Creación de los features por medio de la red VGG16

In [ ]:
batch_size = 16
save_bottlebeck_features()

### Entrenamiento de los Top Models

In [ ]:
batch_size = 1024
epochs = 300
num_classes = 2
train_top_model(1)

In [ ]:
train_top_model(2)

In [ ]:
train_top_model(3)

In [ ]:
train_top_model(4)

In [ ]:
train_top_model(5)

In [ ]:
train_top_model(6)

In [ ]:
train_top_model(7)

In [ ]:
epochs = 800
train_top_model(8)

In [ ]:
epochs = 1500
train_top_model(9)

In [ ]:
epochs = 500
train_top_model(10)

### Resultados de cada modelo

Los resultados de cada modelo se encuentran detallados en este [link](https://docs.google.com/spreadsheets/d/15_S33peWDFdnzXx5T0yxGEaGsqcYP6dQLyk6DUqXo5A/edit#gid=0) en la solapa "Dañado o Sano".

##### El modelo seleccionado como el mejor es el 8
- Acc: 0,8989
- Val Acc: 0,8743
- Loss: 0,3245
- Val Loss: 0,4068

### Fine Tunning

In [ ]:
from keras.models import Model
from keras.models import load_model

top_model_weights_path = 'bottleneck_damaged_or_not.h5'
epochs = 80
batch_size = 32
datagen_train_fine = ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rotation_range=90)

generator_train_fine = datagen_train_fine.flow_from_directory(train_data_dir,  
                                                target_size=(img_width, img_height),  
                                                batch_size=batch_size,  
                                                class_mode='categorical',  
                                                shuffle=False)

nb_train_samples_fine = len(generator_train_fine.filenames)  
num_classes = len(generator_train_fine.class_indices)  

train_data = generator_train_fine
train_labels = generator_train_fine.classes  
train_labels = to_categorical(train_labels, num_classes=num_classes)

generator_validation_fine = datagen_train_fine.flow_from_directory(validation_data_dir,  
                                                        target_size=(img_width, img_height),  
                                                        batch_size=batch_size,  
                                                        class_mode='categorical',  
                                                        shuffle=False)  

nb_validation_samples_fine = len(generator_validation_fine.filenames)

validation_data = nb_validation_samples_fine
validation_labels = generator_validation_fine.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)


base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))

print(base_model.output_shape[1:])

top_model = Sequential()  
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))  
top_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
top_model.add(Dropout(0.5))
top_model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
top_model.add(Dropout(0.5))
top_model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))

top_model.compile(optimizer=optimizers.RMSprop(lr=0.01, decay=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

top_model.load_weights(top_model_weights_path)

model_fine = Model(inputs=base_model.input, outputs=top_model(base_model.output))

for layer in model_fine.layers[:15]:
    print(layer.name)
    layer.trainable = False
model_fine.layers[15].name

model_fine.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

print(model_fine.summary())

model_fine = load_model('MRETZLAF_DAMAGED_OR_NOT_FINE.hdf5')

checkpointer = ModelCheckpoint(monitor='val_acc', filepath='MRETZLAF_DAMAGED_OR_NOT_FINE.hdf5', verbose=1, save_best_only=True)

history_fine = model_fine.fit_generator(generator_train_fine,
                                        steps_per_epoch=nb_train_samples_fine // batch_size,
                                        epochs=epochs,
                                        validation_data=generator_validation_fine,
                                        validation_steps=nb_validation_samples_fine // batch_size,
                                        verbose=1,
                                        callbacks=[checkpointer])

plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.show()

plt.plot(history_fine.history['acc'])
plt.plot(history_fine.history['val_acc'])
plt.show()

### Resultados

**Accuracy**
- Acc: 0,9784
- Val Acc: 0,954
![Accuracy](Fine_acc_red2.png)

**Loss**
- Loss: 0,0952
- Val Loss: 0,1712
![title](Fine_loss_red2.png)

### Testing Data

In [ ]:
from keras.models import load_model

model_fine = load_model('MRETZLAF_DAMAGED_OR_NOT_FINE.hdf5')

test_data_dir = 'C:/data/car-damage/damaged_or_not/testing'
test_datagen = ImageDataGenerator(rescale=1. / 255)
flow_test = test_datagen.flow_from_directory(test_data_dir,  target_size=(img_width, img_height),  batch_size=batch_size,class_mode='categorical')
print(model_fine.metrics_names)
model_fine.evaluate_generator(flow_test,5)

- Test Acc: 0,953135
- Test Loss: 0,2075

## Conclusiones

Con la primer red se obtuvieron muy buenos resultados probando la red en un entorno de testing con datos reales. Se está trabajando para implementarla en producción aunque se debe probar con mas datos reales.

La segunda red no tuvo tan buenos resultados ya que los daños pequeños como rayones o aboyaduras chicas muchas veces no son detectados correctamente. Se está trabajando en un nuevo modelo de detección de daño que permita clasificar el daño en 3 categorias: Leve - Moderado - Grave. Con este enfoque, se trata de obtener un resultado más productivo para el negocio ya que es más fácil detectar daños graves que daños leves y podría hacerse una clasificación automática de los siniestros graves sin necesidad de que intervenga un tasador, dejando el resto de los casos para ser validados por una persona.

## Utilizando la red (Web Service)

Para utilizar la red 1, la disponibilizamos como webservices, recibiendo la url de una imagen a detectar y produciendo una salida json con la probabilidad de que sea un auto.

![Title](webservice.png)